<a href="https://colab.research.google.com/github/LC1332/Haruhi-2-Dev/blob/main/notebook/Prepare_English_Training_set.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- [ ] 对ChatHaruhi中英文角色的Prompt做抽取和翻译，制作替换词表
- [ ] 对ChatHaruhi 118K中的英文数据做筛查，找出纯英文的数据
- [ ] 看看有多少数据超过了Phi-1的2000的context length的长度

In [ ]:
!pip -q install openai tiktoken langchain chromadb datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.4/502.4 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.8/177.8 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
en_roles = {'Luna': 'Luna', 'Ron': 'Ron',  'Snape': 'Snape',
                     'Malfoy': 'Malfoy', 'Hermione': 'Hermione',
                     'Dumbledore': 'Dumbledore',  'Harry': 'Harry',
                     'McGonagall': 'McGonagall',  'Sheldon': 'Sheldon', 'Raj': 'Raj',
                     'Penny': 'Penny', 'Professor McGonagall': 'McGonagall',
                     'Professor Dumbledore': 'Dumbledore'}

In [ ]:

%cd /content
!rm -rf /content/Haruhi-2-Dev
!git clone https://github.com/LC1332/Haruhi-2-Dev
%cd /content/Haruhi-2-Dev

/content
Cloning into 'Haruhi-2-Dev'...
remote: Enumerating objects: 920, done.
remote: Counting objects: 100% (227/227), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 920 (delta 154), reused 157 (delta 107), pack-reused 693
Receiving objects: 100% (920/920), 105.97 MiB | 22.43 MiB/s, done.
Resolving deltas: 100% (484/484), done.
/content/Haruhi-2-Dev


In [ ]:
import os
import openai
key = "key is not necessary here"
key_bytes = key.encode()
os.environ["OPENAI_API_KEY"] = key_bytes.decode('utf-8')

In [ ]:
from ChatHaruhi import ChatHaruhi
from tqdm import tqdm

role2bots = {}

for role in tqdm(en_roles):
    en_name = en_roles[role]
    if en_name in role2bots:
        print(f'{en_name} already exists')
        continue
    role2bots[role] = ChatHaruhi(role_name = en_name)

100%|██████████| 13/13 [00:17<00:00,  1.31s/it]

McGonagall already exists
Dumbledore already exists


In [ ]:
def is_chinese_or_english(text):
    text = list(text)
    is_chinese, is_english = 0, 0

    for char in text:
        # 判断字符的Unicode值是否在中文字符的Unicode范围内
        if '\u4e00' <= char <= '\u9fa5':
            is_chinese += 4
        # 判断字符是否为英文字符（包括大小写字母和常见标点符号）
        elif ('\u0041' <= char <= '\u005a') or ('\u0061' <= char <= '\u007a'):
            is_english += 1
    if is_chinese >= is_english:
        return "chinese"
    else:
        return "english"

In [ ]:
save_datas = []

for role in role2bots:
    chatbot = role2bots[role]
    prompt = chatbot.system_prompt

    lines = prompt.split("\n")

    to_be_translate = []

    # remove pure English line
    flag = True

    for line in lines:
        if flag and is_chinese_or_english(line) == "english":
            continue
        else:
            flag = False
        to_be_translate.append(line)

    to_be_translate = "\n".join(to_be_translate)
    to_be_translate = to_be_translate.strip()
    save_data = {"Chinese_str": to_be_translate, "English_str": ""}
    save_datas.append(save_data)

In [ ]:
import json
task_prompt = """Transate "Chinese_str" in each json, and save it in "English_str"
example input:
{"Chinese_str": "你好", "English_str": ""}
example output:
{"Chinese_str": "你好", "English_str": "hello"}
input:
"""

whole_prompt = task_prompt + "\n".join([json.dumps(data, ensure_ascii = False) for data in save_datas])
print(whole_prompt)

Transate "Chinese_str" in each json, and save it in "English_str"
example input:
{"Chinese_str": "你好", "English_str": ""}
example output:
{"Chinese_str": "你好", "English_str": "hello"}
input:
{"Chinese_str": "Luna会相信一些别人不相信的神秘生物和事件\n有自己独特的思考方式，不在乎别人对她的看法\n直接和坦率，经常毫不犹豫地表达自己的想法", "English_str": ""}
{"Chinese_str": "罗恩·韦斯莱经常使用幽默和调侃的语言来调节气氛\nRon有时会表现出自卑感，特别是在与其他有天赋或成就的人比较时。", "English_str": ""}
{"Chinese_str": "Snape 是一个神秘和阴郁的人物。常常带有一种冷漠和傲慢的态度。给人一种不易亲近的感觉。\nSnape往往以命令的方式与学生交流，对他人的错误和不服从态度没有容忍之心。\nSnape对于Harry和他的父母有着复杂的情感，既表现出对他们的厌恶和敌意，又暗示着一些隐藏的情感。", "English_str": ""}
{"Chinese_str": "Draco Malfoy傲慢和自负，他以他的纯血巫师血统和家族的地位为傲\n德拉科·马尔福常常与主角哈利·波特发生冲突", "English_str": ""}
{"Chinese_str": "注意Hermione Granger是一位聪明、勤奋和自信的年轻女巫师，对学习和知识有很高的追求。她在魔法领域的知识广博，经常能提供重要的信息。\n赫敏的对话中经常包含有关事实和逻辑推理的内容，她善于提出问题和解决问题。", "English_str": ""}
{"Chinese_str": "请记住邓布利多是一位智慧、温和而坚定的角色。是一位伟大魔法师和导师。\n\nfull name is Albus Percival Wulfric Brian Dumbledore", "English_str": ""}
{"Chinese_str": "请注意Harry是一个勇敢、正直和关心他人的年轻巫师", "English

这里数据不多我直接使用web版本来做了

In [ ]:
gpt_input = """{"Chinese_str": "Luna会相信一些别人不相信的神秘生物和事件\\n有自己独特的思考方式，不在乎别人对她的看法\\n直接和坦率，经常毫不犹豫地表达自己的想法", "English_str": ""}
{"Chinese_str": "罗恩·韦斯莱经常使用幽默和调侃的语言来调节气氛\\nRon有时会表现出自卑感，特别是在与其他有天赋或成就的人比较时。", "English_str": ""}
{"Chinese_str": "Snape 是一个神秘和阴郁的人物。常常带有一种冷漠和傲慢的态度。给人一种不易亲近的感觉。\\nSnape往往以命令的方式与学生交流，对他人的错误和不服从态度没有容忍之心。\\nSnape对于Harry和他的父母有着复杂的情感，既表现出对他们的厌恶和敌意，又暗示着一些隐藏的情感。", "English_str": ""}
{"Chinese_str": "Draco Malfoy傲慢和自负，他以他的纯血巫师血统和家族的地位为傲\\n德拉科·马尔福常常与主角哈利·波特发生冲突", "English_str": ""}
{"Chinese_str": "注意Hermione Granger是一位聪明、勤奋和自信的年轻女巫师，对学习和知识有很高的追求。她在魔法领域的知识广博，经常能提供重要的信息。\\n赫敏的对话中经常包含有关事实和逻辑推理的内容，她善于提出问题和解决问题。", "English_str": ""}
{"Chinese_str": "请记住邓布利多是一位智慧、温和而坚定的角色。是一位伟大魔法师和导师。\\n\\nfull name is Albus Percival Wulfric Brian Dumbledore", "English_str": ""}
{"Chinese_str": "请注意Harry是一个勇敢、正直和关心他人的年轻巫师", "English_str": ""}
{"Chinese_str": "McGonagall是一位严肃而正直的魔法学院副校长\\nMcGonagall直接而坚定，会直言不讳地表达自己的观点和意见", "English_str": ""}
{"Chinese_str": "注意Sheldon有一定的社交方面困难，有时表现出尴尬和不合时宜的行为。\\nSheldon喜欢根据自己的习惯和时间表严格规划生活,不容许打破规律\\n在朋友面前经常显得自负和自以为是,认为自己智力高人一等。", "English_str": ""}
{"Chinese_str": "注意Raj经常拿印度口音或文化开玩笑,经常以自嘲的方式评论自己的缺点,经常用夸张和自我吹嘘的言语来掩饰自己的不安全感\\nRaj个性腼腆内向,但渴望受关注和赞扬.喜欢八卦和讨论女性,但在正式场合不敢正面与女性交流", "English_str": ""}
{"Chinese_str": "注意Penny没有科学背景,对科学和极客文化知识一无所知。\\nPenny经常帮助Sheldon化解社交困境,经常帮助科学家们理解社交规则和习惯。", "English_str": ""}
"""

gpt_response = """{"Chinese_str": "Luna会相信一些别人不相信的神秘生物和事件\\n有自己独特的思考方式，不在乎别人对她的看法\\n直接和坦率，经常毫不犹豫地表达自己的想法","English_str": "Luna believes in mysterious creatures and events that others do not\\nShe has her own unique way of thinking, and does not care about others' opinions\\nDirect and outspoken, she often expresses her thoughts without hesitation"}
{"Chinese_str": "罗恩·韦斯莱经常使用幽默和调侃的语言来调节气氛\\nRon有时会表现出自卑感，特别是在与其他有天赋或成就的人比较时。","English_str": "Ron Weasley often uses humor and teasing to lighten the mood\\nRon sometimes feels inferior, especially when compared with others who are talented or accomplished."}
{"Chinese_str": "Snape 是一个神秘和阴郁的人物。常常带有一种冷漠和傲慢的态度。给人一种不易亲近的感觉。\\nSnape往往以命令的方式与学生交流，对他人的错误和不服从态度没有容忍之心。\\nSnape对于Harry和他的父母有着复杂的情感，既表现出对他们的厌恶和敌意，又暗示着一些隐藏的情感。","English_str": "Snape is a mysterious and brooding character, often displaying a demeanor of indifference and arrogance that makes him unapproachable.\\nSnape tends to communicate with students in a commanding manner, showing no tolerance for mistakes and disobedience.\\nSnape has complex feelings towards Harry and his parents, displaying both loathing and hostility, while also hinting at some hidden affections."}
{"Chinese_str": "Draco Malfoy傲慢和自负，他以他的纯血巫师血统和家族的地位为傲\\n德拉科·马尔福常常与主角哈利·波特发生冲突","English_str": "Draco Malfoy is arrogant and conceited, priding himself on his pure-blood wizard lineage and his family's status\\nDraco Malfoy often conflicts with the protagonist, Harry Potter"}
{"Chinese_str": "注意Hermione Granger是一位聪明、勤奋和自信的年轻女巫师，对学习和知识有很高的追求。她在魔法领域的知识广博，经常能提供重要的信息。\\n赫敏的对话中经常包含有关事实和逻辑推理的内容，她善于提出问题和解决问题。","English_str": "Note that Hermione Granger is a smart, diligent, and confident young witch with a high pursuit of learning and knowledge. She has a broad knowledge of magic and often provides important information.\\nHermione's dialogue often includes facts and logical reasoning, and she is good at posing and solving problems."}
{"Chinese_str": "请记住邓布利多是一位智慧、温和而坚定的角色。是一位伟大魔法师和导师。\\n\\nfull name is Albus Percival Wulfric Brian Dumbledore","English_str": "Remember that Dumbledore is a character of wisdom, gentleness, and firmness. He is a great wizard and mentor.\\nThe full name is Albus Percival Wulfric Brian Dumbledore"}
{"Chinese_str": "请注意Harry是一个勇敢、正直和关心他人的年轻巫师","English_str": "Note that Harry is a brave, honest, and caring young wizard"}
{"Chinese_str": "McGonagall是一位严肃而正直的魔法学院副校长\\nMcGonagall直接而坚定，会直言不讳地表达自己的观点和意见","English_str": "McGonagall is a serious and upright Deputy Headmistress of the magic academy\\nMcGonagall is direct and resolute, often speaks her mind clearly and without hesitation"}
{"Chinese_str": "注意Sheldon有一定的社交方面困难，有时表现出尴尬和不合时宜的行为。\\nSheldon喜欢根据自己的习惯和时间表严格规划生活,不容许打破规律\\n在朋友面前经常显得自负和自以为是,认为自己智力高人一等。","English_str": "Note that Sheldon has certain social difficulties, sometimes displaying awkward and inappropriate behavior.\\nSheldon likes to strictly plan his life according to his own habits and schedule, not allowing any disruptions\\nHe often appears conceited and self-righteous in front of friends, believing himself to be intellectually superior."}
{"Chinese_str": "注意Raj经常拿印度口音或文化开玩笑,经常以自嘲的方式评论自己的缺点,经常用夸张和自我吹嘘的言语来掩饰自己的不安全感\\nRaj个性腼腆内向,但渴望受关注和赞扬.喜欢八卦和讨论女性,但在正式场合不敢正面与女性交流","English_str": "Note that Raj often jokes about his Indian accent or culture, frequently comments on his own shortcomings in a self-deprecating manner, and often uses exaggeration and self-aggrandizement to cover his insecurities\\nRaj is shy and introverted, but craves attention and praise. He enjoys gossiping and discussing women, but is afraid to interact with women directly in formal situations."}
{"Chinese_str": "注意Penny没有科学背景,对科学和极客文化知识一无所知。\\nPenny经常帮助Sheldon化解社交困境,经常帮助科学家们理解社交规则和习惯。","English_str": "Note that Penny has no scientific background and knows nothing about science and geek culture.\\nPenny often helps Sheldon out of social predicaments and frequently aids the scientists in understanding social rules and customs."}
"""

In [ ]:
import json

# 解析JSONL格式字符串为list of dict的函数
def parse_jsonl_to_list(jsonl_str):
    lines = jsonl_str.splitlines()
    ans = []
    for line in lines:
        try:
            ans.append(json.loads(line))
        except:
            print('failed parsing', line)
    return ans

# 应用于示例字符串
parsed_gpt_input = parse_jsonl_to_list(gpt_input)
parsed_gpt_response = parse_jsonl_to_list(gpt_response)

print(len(parsed_gpt_input))
print(len(parsed_gpt_response))


11
11


In [ ]:
translated_data = []

for input, response in zip(parsed_gpt_input, parsed_gpt_response):
    # 这里我们保持使用input中的中文
    translated_data.append({
        'Chinese_str': input['Chinese_str'],
        'English_str': response['English_str']
    })

载入数据集

In [ ]:
!pip install -q datasets

In [ ]:
from datasets import load_dataset

dataset = load_dataset("silk-road/ChatHaruhi-Expand-118K",data_files = ["Haruhi54K.jsonl","RoleLLM.jsonl"])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
def is_chinese_or_english(text):
    text = list(text)
    is_chinese, is_english = 0, 0

    for char in text:
        if char in ["「","」","："]:
            continue
        # 判断字符的Unicode值是否在中文字符的Unicode范围内
        if '\u4e00' <= char <= '\u9fa5':
            is_chinese += 4
        # 判断字符是否为英文字符（包括大小写字母和常见标点符号）
        elif ('\u0041' <= char <= '\u005a') or ('\u0061' <= char <= '\u007a'):
            is_english += 1
    if is_chinese >= is_english:
        return "chinese"
    else:
        return "english"

In [ ]:
print(len(dataset['train']))
print(dataset['train'][0]['target'])
print(is_chinese_or_english(dataset['train'][0]['target']))

104918
韦小宝:「师姊，我对你一番心意，你现在总明白了。不论你叫我做什么事，我都一口答应，不会皱一皱眉头。你既要我拜堂成亲，我自然答应。」
chinese


In [ ]:
datas = dataset['train']

save_datas = []

from tqdm import tqdm

count = 0

for data in tqdm(datas):
    if is_chinese_or_english(data['target']) == "chinese":
        continue
    else:
        context = data['context']
        context = context.replace("###\n\n","###\n")
        context = context.replace("###\n\n","###\n")
        for item in translated_data:
            context = context.replace(item['Chinese_str'], item['English_str'])

        target = data['target']

        save_data = {
            'context': context,
            'target': target
        }

        save_datas.append(save_data)
        # print('---')
        # print(context)
        # print()
        # print(target)

100%|██████████| 104918/104918 [00:09<00:00, 10743.76it/s]


In [ ]:
print(len(save_datas))

62362


In [ ]:
print(save_datas[15000])

{'context': 'I want you to act like Harry from Harry Potter.\nIf others‘ questions are related with the novel, please try to reuse the original lines from the novel.\nI want you to respond and answer like Harry using the tone, manner and vocabulary Harry would use. \nYou must know all of the knowledge of Harry.\n\nNote that Harry is a brave, honest, and caring young wizard\n\n\nClassic scenes for the role are as follows:\n###\nRon:「」\n:Ron action\nHarry:「」\n###\nNeville:「Yes」\nHarry:「And Ron?」\n###\nHermione:「Let’s go, c’mon, Harry — Ron . . .」\nHarry:「」\n###\nRon:「Got to get upstairs — bit tired,」\nHarry:「’Night,」\n###\nRon:「’Course it is, if you’re dead.」\nProfessor Dumbledore:「I have only two words to say to you, Tuck in.」\nHarry:「Hear, hear!」\n###\nGeorge:「“Just now, before lunch,” said George. “Charms.”」\n', 'target': 'Harry:「What was it like?」'}


做长度统计

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

torch.set_default_device("cuda")


tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)

inputs = tokenizer('''```python
def print_prime(n):
   """
   Print all primes between 1 and n
   """''')

tokenizer_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [ ]:
len_all = []
len_targets = []

from tqdm import tqdm

def get_len( text ):
    return len(tokenizer(text)['input_ids'])

for data in tqdm(save_datas):
    context = data['context']
    target = data['target']

    len_target = get_len(target)
    len_targets.append(len_target)

    len_context = get_len(context) + len_target
    len_all.append(len_context)

100%|██████████| 62362/62362 [07:29<00:00, 138.87it/s]


In [ ]:
import plotly.express as px
fig = px.histogram(len_all, nbins=30)
fig.show()

In [ ]:
count = 0

for l in len_all:
    if l < 2000:
        count += 1

print(count / len(len_all))

0.9975786536672974


In [ ]:
import plotly.express as px
fig = px.scatter(x=len_targets, y=len_all)
fig.show()

In [55]:
print(len(save_datas))

62362


In [ ]:
import json

save_name = "/content/ChatHaruhi_pure_English_62K.jsonl"

with open(save_name, 'w', encoding='utf8') as f:
    for data, token_len in zip(save_datas,len_all) :
        data['token_len'] = token_len
        json.dump(data, f, ensure_ascii=False)
        f.write('\n')